# Syllabus courses scraper

## Data needed to:
### `Course` model:
- name - name of the course
- field - field of the course 
- semester - `Semester` model
- ECTS - ects points of the course
- test_type - exam/test
- additional_info - information about course
- lecturer

### `Semester` model:
- field_by_year - `FieldByYear` model
- number - number of semester
- ECTS_required - required ects points

### `FieldByYear` model:
- field - `Field`
- start_date -
- end_date -

### `Field` model:
- name -
- faculty - `Faculty`
- formula -
- type - stacjonarne/nie
- description - description
- specialization - 
- G1_subject - 
- G2_subject - 

### `Faculty` model:
- name -
- building -

In [1]:
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver

import pandas as pd
import numpy as np
import time

In [ ]:
faculty_df = pd.DataFrame(columns=['name'])
field_df = pd.DataFrame(columns=['faculty_id', 'name', 'formula', 'type', 'description', 'specialization'])
field_by_year_df = pd.DataFrame(columns=['field_id', 'start_date'])
semester_df = pd.DataFrame(columns=['field_by_year_id', 'ects', 'number'])
course_df = pd.DataFrame(columns=['field_id', 'name', 'semester', 'ects', 'test_type', 'lecturer', 'description'])

base_url = "https://sylabusy.agh.edu.pl/"
driver = webdriver.Chrome()
driver.get(base_url)

fields_types = ['Stacjonarne', 'Niestacjonarne']
FIELDS_TYPES = [fields_types[0]]
fields_level = ['Studia inżynierskie I stopnia', "Studia magisterskie inżynierskie II stopnia", "Studia licencjackie I stopnia", "Studia magisterskie II stopnia", "Studia podyplomowe"]
FIELDS_LEVEL = [fields_level[0], fields_level[1]]


def get_all_starting_years():
    rozpoczecie_ul = driver.find_element(By.XPATH, "//div[contains(., 'Rozpoczęcie studiów')]/following-sibling::ul[1]")
    link_elements = rozpoczecie_ul.find_elements(By.TAG_NAME, "a")
    hrefs = [el.get_attribute("href") for el in link_elements]
    return hrefs

def get_all_faculties_per_year():
    linki_wydzialow = []
    nazwy_wydzialow = []
    wszystkie_wydzialy = driver.find_element(By.XPATH, '/html/body/main/div/div/section/div[2]')
    linki = wszystkie_wydzialy.find_elements(By.CSS_SELECTOR, 'div.elements-department a')
    for link in linki:
        href = link.get_attribute("href")
        if link.text != "Semestr wyrównawczy":
            linki_wydzialow.append(href)
            nazwy_wydzialow.append(link.text)
    return linki_wydzialow, nazwy_wydzialow

def get_all_fields_per_faculty():
    main_content = driver.find_element(By.ID, "main-content")
    links_a = main_content.find_elements(By.CSS_SELECTOR, "li.student-view-content-list-item a")
    kierunki_hrefs = []
    for link in links_a:
        if link.get_attribute("innerText").split(',')[0] in FIELDS_LEVEL and link.get_attribute("innerText").split(',')[1][1:] in FIELDS_TYPES:
            kierunki_hrefs.append(link.get_attribute("href"))
    return kierunki_hrefs

def get_faculty_field_description():
    breadcrumb = driver.find_element(By.CSS_SELECTOR, "nav[aria-label='breadcrumb']")
    items = breadcrumb.find_elements(By.CSS_SELECTOR, "ol.breadcrumb li.breadcrumb-item")
    texts = [item.text.strip() for item in items[1:]]

    subtitle = driver.find_element(By.CSS_SELECTOR, ".section-subtitle")
    subtitle_list = [part.strip() for part in subtitle.text.split(",")]

    container = driver.find_element(By.ID, "nav-tab-info-panel")
    # first_paragraph = container.find_element(By.TAG_NAME, "p")
    # opis_kierunku = first_paragraph.get_attribute("innerText").strip()

    opis_kierunku = container.text.split('Opiekun')[0].strip().split('Zobacz pełny opis kierunku')[0].strip()

    texts.extend(subtitle_list)
    texts.append(opis_kierunku)
    return  texts

def get_semesters() -> tuple[webdriver.Chrome, webdriver.Chrome]:
    semestry = driver.find_element(By.ID, 'pills-tabContent') 
    semestry = semestry.find_elements(By.CSS_SELECTOR, ":scope > div")[1:]
    przyciski_semestry = driver.find_element(By.ID, 'syl-grid-tabs') 
    przyciski_semestry = przyciski_semestry.find_elements(By.TAG_NAME, "button")[1:]
    return semestry, przyciski_semestry

def get_specialization_name(specka: webdriver.Chrome, formula, field_name):
    try:
        nazwa_specki = specka.find_element(By.TAG_NAME, "h2")
    except:
        specialization = ''
        if formula == fields_level[1] or formula == fields_level[3]:
            specialization = field_name
    else:
        nazwa_specki = nazwa_specki.find_element(By.TAG_NAME, 'button')
        nazwa_specki = nazwa_specki.find_elements(By.TAG_NAME, "span")[1]
        specialization = nazwa_specki.get_attribute("textContent").strip()
    return specialization

COURSES_BAD_PREFIXES = ('Język', 'Przedmiot humanistyczny', 'Przedmioty humanistyczne', 'Elective Humanistic',
                        'German', 'Russian', 'Spanish', 'French')

def get_valid_courses_buttons(specka:webdriver.Chrome):
    table = specka.find_element(By.TAG_NAME, 'table')
    tbodys = table.find_elements(By.TAG_NAME, 'tbody')
    valid_buttons = []

    for tbody in tbodys:
        # all_tr = tbody.find_elements(By.TAG_NAME, 'tr')   # tutaj moze pojawic sie odfiltrowanie przedmiotow np. humanistycznych, jezykow.
                                                            # trzeba przejsc przez wszystkie <tr> i sprawdzic czy maja atrybut 'class', jezeli nie to sa 
                                                            # to przedmioty do dodania (za wyjatkie 'Zasady wyboru', ale tam nie ma przycisku, to jest git).
                                                            # Jezeli maja np. 'class=syl-grid-group syl-group-199396-element', to dla drugiego elementu po split()
                                                            # szukamy przycisku z 'id=syl-group-199396', dla niego szukamy nazwy i sprawdzamy czy dodac.
        #TODO: ograniczyc jezyki
        all_buttons = tbody.find_elements(By.TAG_NAME, 'button')
        for course_button in all_buttons:
            if course_button.get_attribute('id') and course_button.get_attribute('aria-controls'):
                continue
            else:
                if course_button.get_attribute('textContent').strip().startswith(COURSES_BAD_PREFIXES):
                    continue
                else:
                    valid_buttons.append(course_button)
    return valid_buttons

def get_course_data(course_button:webdriver.Chrome):
    driver.execute_script("arguments[0].click();", course_button)
    time.sleep(0.5)
    modal = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CLASS_NAME, "modal-content")))
    exit_button = modal.find_element(By.CLASS_NAME, "modal-header").find_element(By.TAG_NAME, "button")

    all_clean_descrpitions = []

    sylabus_content = modal.find_element(By.ID, "syllabus_content")
    divs = sylabus_content.find_elements(By.CSS_SELECTOR, ':scope > div')

    # course_name = divs[0].text[:-23]
    if divs[0].text[-23:] == "Karta opisu przedmiotu":
        course_name = divs[0].text[:-23]
    else:
        course_name = divs[0].text[:-26]
    lecturer = sylabus_content.find_element(By.CLASS_NAME, 'head-author-right')
    lecturer = lecturer.get_attribute('textContent').strip()
    # print(lecturer)

    table = sylabus_content.find_element(By.TAG_NAME, 'table')
    tds = table.find_elements(By.TAG_NAME, 'td')
    test_type_divs = tds[1].find_elements(By.TAG_NAME, 'div')
    test_type = test_type_divs[1].text
    ects = int(tds[2].text.split()[-1])

    tresci_programowe = sylabus_content.find_elements(By.CSS_SELECTOR, 'tbody.syllabus-data')[1]
    rows = tresci_programowe.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        tds = row.find_elements(By.CSS_SELECTOR, ':scope > td')
        td = tds[1]
        raw_text = td.get_attribute('innerText')
        clean_text = ' '.join(raw_text.split())
        all_clean_descrpitions.append(clean_text)
    description = ' '.join(all_clean_descrpitions)
    # print(description)
    driver.execute_script("arguments[0].click();", exit_button)
    return course_name, lecturer, test_type, ects, description


starting_years_hrefs = [base_url]
starting_years_hrefs.extend(get_all_starting_years())

for href in starting_years_hrefs[:1]:
    driver.get(href)
    time.sleep(0.1)
    faculties_hrefs, faculties_names = get_all_faculties_per_year()

    for faculty_href, faculty_name in zip(faculties_hrefs[:1], faculties_names[:1]):
        if faculty_name not in faculty_df['name'].values:
            row = {'name': faculty_name}
            faculty_df = pd.concat([faculty_df, pd.DataFrame([row])], ignore_index=True)

        driver.get(faculty_href)
        time.sleep(0.1)
        fields_hrefs = get_all_fields_per_faculty()

        for field_href in fields_hrefs[:4]:
            driver.get(field_href)
            faculty_name, field_name, starting_year, formula, studies_type, description = get_faculty_field_description()
            semesters, semesters_buttons = get_semesters()

            for idx, (button, semester) in enumerate(zip(semesters_buttons, semesters)):
                semester_num = idx + 1
                driver.execute_script("arguments[0].click();", button)
                specializations = semester.find_elements(By.TAG_NAME, 'article')

                for specka in specializations:
                    specialization_name = get_specialization_name(specka, formula, field_name)
                    if field_name not in field_df['name'].values or specialization_name not in field_df['specialization'].values:
                        faculty_id = faculty_df[faculty_df['name'] == faculty_name].index.values[0]
                        row = {'faculty_id': faculty_id, 'name': field_name, 'formula': formula, 'type': studies_type, 'description': description, 'specialization': specialization_name}
                        field_df = pd.concat([field_df, pd.DataFrame([row])], ignore_index=True)

                    # columns=['field_id', 'name', 'semester', 'ects', 'test_type', 'lecturer', 'description']
                    # course_df
                    courses_buttons = get_valid_courses_buttons(specka=specka)
                    # print(len(courses_buttons))

                    for course_button in courses_buttons:
                        course_name, lecturer, test_type, ects, description = get_course_data(course_button)

                        field_id = field_df[(field_df['name'] == field_name) & (field_df['specialization'] == specialization_name)].index.values[0]
                        row = {'field_id': field_id, 'name': course_name, 'lecturer': lecturer, 'semester': semester_num, 'test_type': test_type, 'ects': ects, 'description': description}
                        course_df = pd.concat([course_df, pd.DataFrame([row])], ignore_index=True)
                    
driver.quit()

Katarzyna Cyran
Zajęcia prowadzone w trybie mieszanym. Deformacje ciągłe i nieciągłe. Przekroje geologiczne i ich interpretacja na podstawie danych z wierceń. Mapy geologiczne, przekroje geologiczne na podstawie map. Elementy zalegania warstw i ich pomiary. Klasyfikacja i rodzaje gruntów. Obecność na ćwiczeniach audytoryjnych obowiązkowa. Zajęcia prowadzone w trybie mieszanym Pochodzenie i budowa Ziemi. Czas geologiczny i wędrówka kontynentów. Tektonika płyt litosfery. Trzęsienia Ziemi: przyczyny, przebieg, skutki. Plutonizm i wulkanizm przyczyny i mechanizm. Metamorfizm. Sedymentacja i środowiska sedymentacji. Diageneza. Wody płynące i podziemne. Działalność lodowców. Wietrzenie i erozja. Powierzchniowe ruchy masowe, denudacja. Budowa geologiczna Polski. Znaczenie gospodarcze skał. Zajęcia prowadzone w trybie stacjonarnym, mogą być prowadzone w trybie mieszanym Minerały skałotórcze. Struktury i tekstury skał magmowych, metamorficznych i osadowych. Opis i rozpoznawanie podstawowych ska

In [17]:
# faculty_df[faculty_df['nazwa'] == 'Wydział Odlewnictwa'].index.values[0]
# field_df
# faculty_df
course_df

,field_id,name,semester,ects,test_type,lecturer,description
0,0,Geolo,1,3,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,0,Geometria wykreś,1,2,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,0,Grafika inżynierska i rysunek technic,1,3,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,0,Geode,1,3,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,0,Fizyk,1,3,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...
...,...,...,...,...,...,...,...
455,9,Hydraulics of Water Well,7,3,Completing the classes,"Radosław Pomykała, Krzyszt...",1. Well Hydraulics (3) 1.1. Concept of head 1....
456,9,Basics of Circular Econom,7,3,Completing the classes,Radosław Pomykała,Current trends in development and changes in t...
457,9,Mining & Econom,7,3,Completing the classes,Barbara Kowal,An organizational matters – An overview of Pol...
458,9,Applied Geomechanic,7,3,Completing the classes,"Agnieszka Stopkowicz, Mare...",Application of closed-form solutions for solvi...


In [ ]:
df = pd.DataFrame()

def extract_info_from_current_page():
    global df
    print("===> Jestem na stronie:", driver.current_url)

    wydzial_links = []
    wszystkie_wydzialy = driver.find_element(By.XPATH, '/html/body/main/div/div/section/div[2]')
    linki = wszystkie_wydzialy.find_elements(By.CSS_SELECTOR, 'div.elements-department a')

    for link in linki:
        href = link.get_attribute("href")
        if href and href.startswith("/"):
            href = base_url + href
        print("Znaleziony link:", href)
        wydzial_links.append(href)
    
    for href in wydzial_links:
        driver.get(href)
        # time.sleep(0.1)

        nazwa_wydzialu = driver.find_element(By.TAG_NAME, 'h1').get_attribute('innerText')
        print(nazwa_wydzialu)
        if nazwa_wydzialu == "Semestr wyrównawczy":
            continue

        wszytskie_kierunki = driver.find_element(By.ID, "main-content")

        links = wszytskie_kierunki.find_elements(By.CSS_SELECTOR, "li.student-view-content-list-item a")

        linki_kierunkow = [link.get_attribute("href") for link in links 
                           if (link.get_attribute("innerText").split(',')[0] == "Studia inżynierskie I stopnia") 
                            or (link.get_attribute("innerText").split(',')[0] == "Studia licencjackie I stopnia")]
        # print(linki_kierunkow)
        # warunek IF wewnatrz List Comprehension ogranicza kierunki tylko do 1 stopnia

        for l in linki_kierunkow:
            metadane = {}
            driver.get(l)
            breadcrumb = driver.find_element(By.CSS_SELECTOR, "nav[aria-label='breadcrumb']")
            items = breadcrumb.find_elements(By.CSS_SELECTOR, "ol.breadcrumb li.breadcrumb-item")
            texts = [item.text.strip() for item in items[1:]]

            metadane['wydzial'] = texts[0]
            metadane['kierunek'] = texts[1]

            subtitle = driver.find_element(By.CSS_SELECTOR, ".section-subtitle")
            subtitle_list = [part.strip() for part in subtitle.text.split(",")]

            metadane['rok rozpoczecia'] = subtitle_list[0]
            metadane['typ 1'] = subtitle_list[1]
            metadane['typ 2'] = subtitle_list[2]

            container = driver.find_element(By.ID, "nav-tab-info-panel")
            first_paragraph = container.find_element(By.TAG_NAME, "p")
            opis_kierunku = first_paragraph.get_attribute("innerText").strip()

            metadane['opis kierunku'] = opis_kierunku
            # print(metadane)


#TODO: 
            semestry = driver.find_element(By.ID, 'pills-tabContent') 
            semestry = semestry.find_elements(By.CSS_SELECTOR, ":scope > div")[1:]

            przyciski_semestry = driver.find_element(By.ID, 'syl-grid-tabs') 
            przyciski_semestry = przyciski_semestry.find_elements(By.TAG_NAME, "button")[1:]
            
            for idx, (przycisk, semester) in enumerate(zip(przyciski_semestry, semestry)):
                metadane['semester'] = idx + 1
                # przycisk.click()
                driver.execute_script("arguments[0].click();", przycisk)

                specjalizacje = semester.find_elements(By.TAG_NAME, 'article')

                for specka in specjalizacje:
                    try:
                        nazwa_specki = specka.find_element(By.TAG_NAME, "h2")
                    except:
                        metadane['specka'] = None
                    else:
                        nazwa_specki = nazwa_specki.find_element(By.TAG_NAME, 'button')
                        nazwa_specki = nazwa_specki.find_elements(By.TAG_NAME, "span")[1]
                        metadane['specka'] = nazwa_specki.get_attribute("textContent").strip()

                    # print(metadane)
                    przedmioty = []
                    opisy = []
                    tablica = specka.find_element(By.TAG_NAME, 'table')

                    tablice = tablica.find_elements(By.TAG_NAME, 'tbody') # do zwyklego i obierakow

                    for tbody in tablice:
                        pr, o = go_through_tbody_element(tbody)
                        przedmioty.extend(pr)
                        opisy.extend(o)
                    # print(przedmioty)

                    for (p, o) in zip(przedmioty, opisy):
                        row = metadane.copy()
                        row['Przedmiot'] = p
                        row['Opis przedmiotu'] = o
                        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
                    
                    # print(df.tail(1))
        print('koniec wydzialu')


def go_through_tbody_element(element: webdriver.Chrome):
    przedmioty = []
    opisy_przedmiotow = []
    wiersze = element.find_elements(By.CSS_SELECTOR, ":scope > tr")

    for wiersz in wiersze:
        przedmiot = wiersz.find_element(By.TAG_NAME, "td")
        skip = False
        try:
            przedmiot = przedmiot.find_element(By.TAG_NAME, "button")
            nazwa_przedmiotu = przedmiot.get_attribute("textContent").strip()
        except:
            try:
                przedmiot = przedmiot.find_element(By.TAG_NAME, "a")
                nazwa_przedmiotu = przedmiot.get_attribute("textContent").strip()
            except:
                nazwa_przedmiotu = driver.execute_script("""let element = arguments[0];
                                                            let text = "";
                                                            for (let node of element.childNodes) {
                                                                if (node.nodeType === Node.TEXT_NODE) {
                                                                    text += node.textContent;
                                                                }
                                                            }
                                                            return text.trim();
                                                        """, przedmiot)
                skip = True
        if skip:
            przedmioty.append(nazwa_przedmiotu)
            tresci_programowe = 'None'
            opisy_przedmiotow.append(tresci_programowe)

        elif nazwa_przedmiotu.startswith("Wychowanie"):
            przedmioty.append(nazwa_przedmiotu)
            tresci_programowe = 'Wychowanie fizyczne'
            opisy_przedmiotow.append(tresci_programowe)
            print(tresci_programowe)

        elif not (nazwa_przedmiotu.startswith("Język") or nazwa_przedmiotu.startswith("Zasady wyboru") \
                or nazwa_przedmiotu.startswith("H. Przed") or nazwa_przedmiotu.startswith("Przedmioty") \
                or nazwa_przedmiotu.startswith("O. Przed") or nazwa_przedmiotu.startswith("Moduł") \
                or nazwa_przedmiotu.startswith("Student") or nazwa_przedmiotu.startswith("J. J") \
                or nazwa_przedmiotu.startswith("Przedmiot UBPO") or nazwa_przedmiotu.startswith("Przedmiot obco") \
                or nazwa_przedmiotu == " " or nazwa_przedmiotu == "" or nazwa_przedmiotu == "\n" \
                or nazwa_przedmiotu.startswith("A. J") or nazwa_przedmiotu.startswith("A. Przed") \
                or nazwa_przedmiotu.startswith("H. J") or nazwa_przedmiotu.startswith("O. J") \
                or nazwa_przedmiotu.startswith("J. Przed") or ("Przedmiot" in nazwa_przedmiotu)\
                or nazwa_przedmiotu.startswith("A.") or nazwa_przedmiotu.startswith("H.") \
                or 'Dyplomowania' in nazwa_przedmiotu or nazwa_przedmiotu.startswith("D.") \
                or 'dyplomowania' in nazwa_przedmiotu or nazwa_przedmiotu.startswith("O.") \
                or nazwa_przedmiotu.startswith("Grupa") or nazwa_przedmiotu.startswith("Grupy")\
                or nazwa_przedmiotu.startswith("Systemy ekspertowe i sztuczna inteligencja")\
                or nazwa_przedmiotu.startswith("Modelowanie i symulacje")\
                or nazwa_przedmiotu.startswith("Analiza obrazów i projektowanie wspomagane komputerowo") \
                or 'obieraln' in nazwa_przedmiotu or nazwa_przedmiotu.startswith("Warsztaty") \
                or nazwa_przedmiotu.startswith("Ścieżk")):
            przedmioty.append(nazwa_przedmiotu)
            # przedmiot

            driver.execute_script("arguments[0].click();", przedmiot)
            time.sleep(0.5)

            # Poczekaj aż modal będzie widoczny
            print(nazwa_przedmiotu)
            modal = WebDriverWait(driver, 1).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "modal-content"))
            )

            # Teraz pobierz przycisk zamknięcia
            exit_button = modal.find_element(By.CLASS_NAME, "modal-header").find_element(By.TAG_NAME, "button")


            content = modal.find_element(By.ID, 'list_dialog_body')
            sylabus_content = content.find_element(By.ID, "syllabus_content")
            # print(len(sylabus_content))
            sylabus_data = sylabus_content.find_elements(By.CSS_SELECTOR, "tbody.syllabus-data")[1]

            tresci_programowe = ''

            wiersze = sylabus_data.find_elements(By.TAG_NAME, 'tr')
            opisy_programu = [i.find_elements(By.CSS_SELECTOR, ":scope > td")[1] for i in wiersze]

            for opis in opisy_programu:
                paragrafy = opis.find_elements(By.TAG_NAME, 'p')
                for paragraf in paragrafy:
                    tresci_programowe = tresci_programowe + paragraf.get_attribute('textContent')

            # Poczekaj aż przycisk będzie klikalny i kliknij
            # WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn-close"))).click()

            opisy_przedmiotow.append(tresci_programowe)

            driver.execute_script("arguments[0].click();", exit_button)
            print(tresci_programowe)

    return przedmioty, opisy_przedmiotow
    


# Start
driver = webdriver.Chrome()
driver.get("https://sylabusy.agh.edu.pl/")

# 1. Wykonaj akcje na pierwszej stronie
extract_info_from_current_page()


# 2. Pobierz hrefy do dalszych stron
rozpoczecie_ul = driver.find_element(By.XPATH, "//div[contains(., 'Rozpoczęcie studiów')]/following-sibling::ul[1]")
link_elements = rozpoczecie_ul.find_elements(By.TAG_NAME, "a")
hrefs = [el.get_attribute("href") for el in link_elements]

# # 3. Przejdź przez pozostałe linki
# for href in hrefs:
#     driver.get(href)
#     time.sleep(0.1)  # ewentualnie WebDriverWait
#     extract_info_from_current_page()
#     # break

df.to_excel('sylabus.xlsx', index=False)

driver.quit()

===> Jestem na stronie: https://sylabusy.agh.edu.pl/
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/2
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/3
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/16
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/4
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/5
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/6
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/7
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/8
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/9
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/10
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/11
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/12
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/13
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/17
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/14
Znaleziony link: https://

TimeoutException: Message: 


### 6 kierunkow i tylko jeden rocznik

In [ ]:
df = pd.DataFrame()

def extract_info_from_current_page():
    global df

    wydzial = driver.find_element(By.XPATH, '/html/body/main/div/div/section/div[2]')
    wydzial_link = wydzial.find_elements(By.CSS_SELECTOR, 'div.elements-department a')
    linki = wydzial_link[:1] + wydzial_link[2:7]
    wydzial_links = []

    for link in linki:
        href = link.get_attribute("href")
        wydzial_links.append(href)
    
    for href in wydzial_links:
        driver.get(href)

        # nazwa_wydzialu = driver.find_element(By.TAG_NAME, 'h1').get_attribute('innerText')
        
        wszytskie_kierunki = driver.find_element(By.ID, "main-content")
        pierwszy_kierunek = wszytskie_kierunki.find_element(By.CSS_SELECTOR, "li.student-view-content-list-item a")
        pierwszy_kierunek_link = pierwszy_kierunek.get_attribute("href")

        metadane = {}
        driver.get(pierwszy_kierunek_link)

        # Naglowek kierunku
        breadcrumb = driver.find_element(By.CSS_SELECTOR, "nav[aria-label='breadcrumb']")
        items = breadcrumb.find_elements(By.CSS_SELECTOR, "ol.breadcrumb li.breadcrumb-item")
        texts = [item.text.strip() for item in items[1:]]

        metadane['wydzial'] = texts[0]
        metadane['kierunek'] = texts[1]

        # Podnaglowek - rok/rok typ studiow, stacjo
        subtitle = driver.find_element(By.CSS_SELECTOR, ".section-subtitle")
        subtitle_list = [part.strip() for part in subtitle.text.split(",")]

        metadane['rok_rozpoczecia'] = subtitle_list[0]
        metadane['typ_studiow'] = subtitle_list[1]
        metadane['forma'] = subtitle_list[2]

        # Opis kierunku studiów
        container = driver.find_element(By.ID, "nav-tab-info-panel")
        first_paragraph = container.find_element(By.TAG_NAME, "p")
        opis_kierunku = first_paragraph.get_attribute("innerText").strip()

        metadane['opis_kierunku'] = opis_kierunku

        # Zbiór wszystkich semestrów
        semestry = driver.find_element(By.ID, 'pills-tabContent') 
        semestry = semestry.find_elements(By.CSS_SELECTOR, ":scope > div")[1:]

        przyciski_semestry = driver.find_element(By.ID, 'syl-grid-tabs') 
        przyciski_semestry = przyciski_semestry.find_elements(By.TAG_NAME, "button")[1:]

        # Przejscie po wszystkich kierunkach
        for idx, (przycisk, semester) in enumerate(zip(przyciski_semestry, semestry)):
            metadane['semester'] = idx + 1

            driver.execute_script("arguments[0].click();", przycisk)

            # Aktualnie nie ma specjalizacji, ale wystepuja pod tym samym elementem, wiec teraz mozna stworzyc liste i wziac 1 element
            specjalizacje = semester.find_elements(By.TAG_NAME, 'article')

            # Przejscie po speckach
            for specka in specjalizacje:
                # Sprawdzenie czy istnieja specki, jesli nie to None,
                # Moze przyda sie w przyszlosci, pod katem pomocy w wyborze studiow magisterskich
                try:
                    nazwa_specki = specka.find_element(By.TAG_NAME, "h2")
                except:
                    metadane['specka'] = 'None'
                else:
                    nazwa_specki = nazwa_specki.find_element(By.TAG_NAME, 'button')
                    nazwa_specki = nazwa_specki.find_elements(By.TAG_NAME, "span")[1]
                    metadane['specka'] = nazwa_specki.get_attribute("textContent").strip()

                # Listy przedmiotow i przynalezacych do nich opisow
                przedmioty = []
                opisy = []

                # Wyszukiwanie wszystkich linkow. Problematyczne przy rozwijanych elementow, takich jak
                # jezyki obce, przedmioty humanistyczne, obieralne, bloki kierunkowe. Tu ewentualnie mozna 
                # by sprobowac brac elementy iteracyjnie po poziomach, ale pojawia sie problem bo rozwiniecie przedmiotow 
                # obieralnych widnieje w elemencie jeden poziom wyzej, a np jezyk obcy tez jest w osobnym elemencie.
                # Wobec tego aktualnie jest zaharcodowane pomijanie rozwiniec
                tablica = specka.find_element(By.TAG_NAME, 'table')
                tablice = tablica.find_elements(By.TAG_NAME, 'tbody')

                # iteracja po tbody (obowiazkowe przedmioty kierunkowe / przedmioty kierunkowe obieralne / humany / jezyki)
                for tbody in tablice:
                    pr, o = go_through_tbody_element(tbody)
                    przedmioty.extend(pr)
                    opisy.extend(o)

                # Dodanie elementow do DataFrame'a
                for (p, o) in zip(przedmioty, opisy):
                    row = metadane.copy()
                    row['Przedmiot'] = p
                    row['Opis przedmiotu'] = o
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

# Funkcja do scrapowania danych z tbody
def go_through_tbody_element(element: webdriver.Chrome):
    przedmioty = []
    opisy_przedmiotow = []

    # Wyszukiwanie wszystkich wierszy (tr) tylko jeden pooziom nizej niz obecnie
    wiersze = element.find_elements(By.CSS_SELECTOR, ":scope > tr")

    for wiersz in wiersze:
        przedmiot = wiersz.find_element(By.TAG_NAME, "td")
        skip = False
        try: # sprawdzenie czy jest przycisk (albo do rozwiniecia, albo do klikniecia)
            przedmiot = przedmiot.find_element(By.TAG_NAME, "button")
            nazwa_przedmiotu = przedmiot.get_attribute("textContent").strip()
        except:
            try: # Czasami nazwa przedmiotu znajduje sie pod tagiem 'a' a nie w 'button'
                przedmiot = przedmiot.find_element(By.TAG_NAME, "a")
                nazwa_przedmiotu = przedmiot.get_attribute("textContent").strip()
            except:
                nazwa_przedmiotu = driver.execute_script("""let element = arguments[0];
                                                            let text = "";
                                                            for (let node of element.childNodes) {
                                                                if (node.nodeType === Node.TEXT_NODE) {
                                                                    text += node.textContent;
                                                                }
                                                            }
                                                            return text.trim();
                                                        """, przedmiot)
                skip = True
        # Jezeli nie ma przycisku/linku to pomija sie
        if skip:
            przedmioty.append(nazwa_przedmiotu)
            tresci_programowe = 'None'
            opisy_przedmiotow.append(tresci_programowe)

        # Jezeli nie jest to rozwiniecie
        elif not (nazwa_przedmiotu.startswith("Język") or nazwa_przedmiotu.startswith("H. Przedmioty") \
                or nazwa_przedmiotu.startswith("Przedmioty") or nazwa_przedmiotu.startswith("O. Przedmioty")
                or nazwa_przedmiotu.startswith("Przedmiot") or nazwa_przedmiotu.startswith("Blok") \
                or nazwa_przedmiotu.startswith("Praktyka") or nazwa_przedmiotu.startswith("Projekt") \
                or nazwa_przedmiotu.startswith("Moduł") or nazwa_przedmiotu.startswith("Grupa") \
                or nazwa_przedmiotu.startswith("Zasady")):
            przedmioty.append(nazwa_przedmiotu)

            driver.execute_script("arguments[0].click();", przedmiot)
            time.sleep(0.8)

            # Poczekaj aż modal będzie widoczny
            print(nazwa_przedmiotu)
            modal = WebDriverWait(driver, 1).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "modal-content")))

            # Pobierz przycisk zamknięcia
            exit_button = modal.find_element(By.CLASS_NAME, "modal-header").find_element(By.TAG_NAME, "button")

            # Znalezienie elementu 'tresci programowe'
            content = modal.find_element(By.ID, 'list_dialog_body')
            sylabus_content = content.find_element(By.ID, "syllabus_content")
            sylabus_data = sylabus_content.find_elements(By.CSS_SELECTOR, "tbody.syllabus-data")[1]

            tresci_programowe = ''

            wiersze = sylabus_data.find_elements(By.TAG_NAME, 'tr')
            opisy_programu = [i.find_elements(By.CSS_SELECTOR, ":scope > td")[1] for i in wiersze]

            # Przejscie po opisach
            for opis in opisy_programu:
                paragrafy = opis.find_elements(By.TAG_NAME, 'p')
                for paragraf in paragrafy:
                    tresci_programowe = tresci_programowe + paragraf.get_attribute('textContent')

            opisy_przedmiotow.append(tresci_programowe)

            driver.execute_script("arguments[0].click();", exit_button)

    return przedmioty, opisy_przedmiotow
    


# Start
driver = webdriver.Chrome()
driver.get("https://sylabusy.agh.edu.pl/")

# 1. Wykonaj akcje na pierwszej stronie (2025/2026)
extract_info_from_current_page()

df.to_excel('sylabus_2025_6_kierunkow.xlsx', index=False)

driver.quit()

Geologia
Geometria wykreślna
Grafika inżynierska i rysunek techniczny
Geodezja
Fizyka I
Chemia (budowlana)
Matematyka 1
Technologie informacyjne
Mechanika teoretyczna
Ochrona własności intelektualnej
Materiały budowlane
Fizyka II
Budownictwo ogólne
Matematyka 2
Eksploracja podwodna
Historia i tradycje górnictwa
Wszechświat, początek -ewolucja - człowiek
Rysunek odręczny dla inżynierów
Budownictwo ogólne
Podstawy architektury i urbanistyki
Fizyka budowli
Mechanika gruntów
Mechanika teoretyczna
Technologia betonu
Wytrzymałość materiałów
Hydraulika i hydrologia
Komputerowe wspomaganie projektowania
Własności skał
Warunki ochrony przeciwpożarowej obiektów budowlanych
Metody obliczeniowe
Fundamentowanie
Podstawy projektowania konstrukcji budowlanych
Mechanika budowli
Budownictwo komunikacyjne
Instalacje budowlane
Wytrzymałość materiałów
Podstawy techniki strzelniczej
BIOZ i ergonomia w budownictwie
Geomechanika
Maszyny budowlane
Konstrukcje murowe i drewniane
Ekonomika budownictwa
Technolog